# CHAPTER 7

## 7-1 전체 구조

## 7-2 합성곱 계층

### 7-2-1 완전연결 계층의 문제점

### 7-2-2 합성곡 연산

### 7-2-3 패딩

### 7-2-4 스트라이드

### 7-2-5 3차원 데이터의 합성곱 연산

### 7-2-6 블록으로 생각하기

### 7-2-7 배치 처리

## 7-3 풀링 계층

### 7-3-1 풀링 계층의 특징

## 7-4 합성곱 / 풀링 계층 구현하기

### 7-4-1 4차원 배열

### 7-4-2 im2col로 데이터 전개하기

### 7-4-3 합성곱 계층 구현하기

In [2]:
import  sys, os
sys.path.append('/content/drive/MyDrive/AI/독학/밑시딥/dataset')
sys.path.append('/content/drive/MyDrive/AI/독학/밑시딥/common')
sys.path.append('/content/drive/MyDrive/AI/독학/밑시딥/')
from common.util import im2col
import numpy as np

In [4]:
x1 = np.random.rand(1 , 3 , 7 , 7)  #(데이터 수 , 채널 수 , 높이 , 너비)
col1 = im2col(x1 , 5 , 5 , stride = 1 , pad = 0)
print(col1.shape)

x2 = np.random.rand(10 , 3, 7, 7)
col2 = im2col(x2 , 5 , 5 , stride = 1 , pad = 0)
print(col2.shape)

(9, 75)
(90, 75)


In [5]:
class Convolution:
    def __init__(self , W , b , stride = 1 , pad = 0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self , x):
        FN ,C , FH , FW = self.W.shape
        N ,C ,H , W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

        col = im2col(x , FH , FW , self.stride, self.pad)
        col_W = self.W.reshape(FN , -1).T
        out = np.dot(col , col_W) + self.b

        out = out.reshape(N , out_h , out_w , -1).transpose(0, 3, 1 , 2)

        return out

### 7-4-4 풀링 계층 구현하기

In [6]:
class Pooling:
    def __init__(self , pool_h , pool_w , stride = 1 , pad = 0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self , x):
        N, C ,H ,W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        #전개 (1)
        col = im2col(x, self.pool_h , self.pool_w , self.stride , self.pad)
        col = col.reshape(-1 , self.pool_h * self.pool_w)

        #최댓값 (2)
        out = np.max(col , axis = 1)

        #성형 (3)
        out = out.reshape(N , out_h , out_w , C).transpose( 0, 3, 1 ,2)

        return out

## 7-5 CNN 구현하기

In [9]:
from collections import OrderedDict
from common.functions import *

class SimpleConvNet:
    def __init__(self , input_dim = (1, 28 , 28) , conv_param = {'filter_num' : 30  , 'filter_size' : 5,
                                                                 'pad' : 0 , 'stride' : 1},
                 hidden_size = 100 , output_size = 10 , weight_init_std = 0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size / 2) * (conv_output_size / 2 ))

        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num , input_dim[0] , filter_size  , filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size , hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size , output_size)
        self.params['b3'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],
                                           self.params['b1'],
                                           conv_param['stride'],
                                           conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(poo_h = 2 , pool_w=2 , stride = 2)
        self.layers['Affine1'] = Affine(self.params['W2'] , self.parmas['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'] , self.parmas['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self , x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    def loss(self , x , t):
        y= self.predict(x)
        return self.last_layer.forward( y, t)

    def gradient(self , x , t):
        #순전파
        self.loss(x,t)

        #역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        #결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db



## 7-6 CNN 시각화하기

### 7-6-1 1번째 층의 가중치 시각화하기

### 7-6-2 층 깊이에 따른 추출 정보 변화

## 7-7 대표적인 CNN

### 7-7-1 LeNet

### 7-7-2 AlexNet